In [2]:
import torch
import torch.nn as nn
from ultralytics import YOLO

In [2]:
import torch
import torch.nn as nn
from ultralytics import YOLO

# Определяем кастомные слои
class PConv(nn.Module):
    """Partial Convolution (PConv)"""
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False):
        super(PConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=bias)
        self.mask_conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x, mask=None):
        if mask is None:
            mask = torch.ones_like(x)
        x = self.conv(x * mask)
        mask = self.mask_conv(mask)
        mask = (mask > 0).float()
        x = self.bn(x)
        x = self.relu(x)
        return x, mask

class EMA(nn.Module):
    """Efficient Multi-scale Attention (EMA)"""
    def __init__(self, in_channels):
        super(EMA, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels // 2, kernel_size=1)
        self.conv2 = nn.Conv2d(in_channels // 2, in_channels, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(in_channels)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        attention = self.conv1(x)
        attention = self.conv2(attention)
        attention = self.bn(attention)
        attention = self.sigmoid(attention)
        return x * attention

# Добавляем кастомные слои в пространство имен глобально
globals()["PConv"] = PConv
globals()["EMA"] = EMA



In [7]:
print(globals()["PConv"])  # Должно вывести <class '__main__.PConv'> 

<class '__main__.PConv'>


In [16]:
# Определяем кастомные слои
class PConv(nn.Module):
    """Partial Convolution (PConv)"""
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False):
        super(PConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=bias)
        self.mask_conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x, mask=None):
        if mask is None:
            mask = torch.ones_like(x)
        x = self.conv(x * mask)
        mask = self.mask_conv(mask)
        mask = (mask > 0).float()
        x = self.bn(x)
        x = self.relu(x)
        return x, mask
    
nn.PConv = PConv

In [ ]:
class PConv(nn.Module):
    def __init__(self, dim, ouc, n_div=4, forward='split_cat'):
        super().__init__()
        self.dim_conv3 = dim // n_div
        self.dim_untouched = dim - self.dim_conv3
        self.partial_conv3 = nn.Conv2d(self.dim_conv3, self.dim_conv3, 3, 1, 1, bias=False)
        self.conv = nn.Conv(dim, ouc, k=1)

        if forward == 'slicing':
            self.forward = self.forward_slicing
        elif forward == 'split_cat':
            self.forward = self.forward_split_cat
        else:
            raise NotImplementedError

    def forward_slicing(self, x): 
        x = x.clone()    
        x[:, :self.dim_conv3, :, :] = self.partial_conv3(x[:, :self.dim_conv3, :, :])
        x = self.conv(x)
        return x

    def forward_split_cat(self, x): 
        x1, x2 = torch.split(x, [self.dim_conv3, self.dim_untouched], dim=1)
        x1 = self.partial_conv3(x1)
        x = torch.cat((x1, x2), 1)
        x = self.conv(x)
        return x

In [9]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'from ultralytics import YOLO',
  "class PConv(nn.Module):\n    def __init__(self, dim, ouc, n_div=4, forward='split_cat'):\n        super().__init__()\n        self.dim_conv3 = dim // n_div\n        self.dim_untouched = dim - self.dim_conv3\n        self.partial_conv3 = nn.Conv2d(self.dim_conv3, self.dim_conv3, 3, 1, 1, bias=False)\n        self.conv = Conv(dim, ouc, k=1)\n\n        if forward == 'slicing':\n            self.forward = self.forward_slicing\n        elif forward == 'split_cat':\n            self.forward = self.forward_split_cat\n        else:\n            raise NotImplementedError\n\n    def forward_slicing(self, x): \n        x = x.clone()    \n        x[:, :self.dim_conv3, :, :] = self.partial_

In [37]:

# Теперь загружаем кастомную модель YOLO с измененной архитектурой
model = YOLO("nets/YOLO/yolo11n.yaml" )  # Используем новый конфиг

In [ ]:
model

<bound method Model.info of YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(144, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(192, eps=0.001, momentum=0.

In [33]:
model

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(144, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(192, eps=0.001, momentum=0.03, affine=True, track_runni

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob

# Функция для загрузки логов PyTorch Lightning
def load_logs(log_dir="lightning_logs"):
    log_files = sorted(glob.glob(f"{log_dir}/version_*/metrics.csv"), key=os.path.getctime)
    
    if not log_files:
        raise FileNotFoundError("Не найдено файлов с логами. Запустите обучение.")
    
    log_file = log_files[-1]  # Берём самый последний лог
    df = pd.read_csv(log_file)
    return df

# Загружаем данные
df = load_logs()

# Построение графиков
plt.figure(figsize=(12, 5))

# Лосс
plt.subplot(1, 2, 1)
plt.plot(df["epoch"], df["train_loss"], label="Train Loss", color="blue")
plt.plot(df["epoch"], df["val_loss"], label="Validation Loss", color="red")
plt.xlabel("Эпоха")
plt.ylabel("Loss")
plt.title("Изменение Loss во время обучения")
plt.legend()

# Точность (val_mAP)
plt.subplot(1, 2, 2)
plt.plot(df["epoch"], df["val_mAP"], label="Validation mAP", color="green")
plt.xlabel("Эпоха")
plt.ylabel("mAP")
plt.title("Изменение точности (mAP) на валидации")
plt.legend()

plt.show()